In [1]:
# Imports
%pip install optuna
%pip install xgboost
%pip install pandas
%pip install scikit-learn
%pip install lightgbm
import optuna
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, RobustScaler, PolynomialFeatures, PowerTransformer, QuantileTransformer, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, BayesianRidge, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import numpy as np
import xgboost as xgb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
df = pd.read_csv('../Diamonds_EDA-main/data_diamonds.csv', sep = ';')

In [33]:
df.head()

,city,price,carat,depth,table,x,y,z,clarity,color,cut
0,Amsterdam,3733,1.01,65.2,56.0,6.23,6.19,4.05,VS2,H,Fair
1,Amsterdam,9452,1.51,65.0,64.0,7.10,7.03,4.59,VS2,H,Fair
2,Zurich,12765,2.00,65.2,56.0,7.92,7.87,5.15,VS2,H,Fair
3,Zurich,17713,1.99,60.5,63.0,8.20,8.11,4.93,VS2,H,Fair
4,Las Vegas,2268,0.70,65.0,56.0,5.60,5.50,3.61,VS2,H,Fair


In [34]:
df = df.drop('city', axis = 1)
df

,price,carat,depth,table,x,y,z,clarity,color,cut
0,3733,1.01,65.2,56.0,6.23,6.19,4.05,VS2,H,Fair
1,9452,1.51,65.0,64.0,7.10,7.03,4.59,VS2,H,Fair
2,12765,2.00,65.2,56.0,7.92,7.87,5.15,VS2,H,Fair
3,17713,1.99,60.5,63.0,8.20,8.11,4.93,VS2,H,Fair
4,2268,0.70,65.0,56.0,5.60,5.50,3.61,VS2,H,Fair
...,...,...,...,...,...,...,...,...,...,...
40450,3193,0.90,57.4,61.0,6.42,6.37,3.67,VVS2,J,Good
40451,15898,2.03,63.2,60.0,7.88,7.94,5.00,VVS2,J,Good
40452,666,0.38,63.4,56.0,4.56,4.62,2.91,VVS2,J,Good
40453,1945,0.91,63.1,57.0,6.06,6.12,3.84,I1,J,Good


In [35]:
df

,price,carat,depth,table,x,y,z,clarity,color,cut
0,3733,1.01,65.2,56.0,6.23,6.19,4.05,VS2,H,Fair
1,9452,1.51,65.0,64.0,7.10,7.03,4.59,VS2,H,Fair
2,12765,2.00,65.2,56.0,7.92,7.87,5.15,VS2,H,Fair
3,17713,1.99,60.5,63.0,8.20,8.11,4.93,VS2,H,Fair
4,2268,0.70,65.0,56.0,5.60,5.50,3.61,VS2,H,Fair
...,...,...,...,...,...,...,...,...,...,...
40450,3193,0.90,57.4,61.0,6.42,6.37,3.67,VVS2,J,Good
40451,15898,2.03,63.2,60.0,7.88,7.94,5.00,VVS2,J,Good
40452,666,0.38,63.4,56.0,4.56,4.62,2.91,VVS2,J,Good
40453,1945,0.91,63.1,57.0,6.06,6.12,3.84,I1,J,Good


In [36]:
df[df['x'] == 0]

,price,carat,depth,table,x,y,z,clarity,color,cut
560,6381,1.14,57.5,67.0,0.0,0.00,0.0,VS1,G,Fair
15694,4954,1.07,61.6,56.0,0.0,6.62,0.0,SI2,F,Ideal
18395,18034,2.25,62.8,59.0,0.0,0.00,0.0,SI2,H,Premium
39897,2130,0.71,64.1,60.0,0.0,0.00,0.0,SI2,F,Good
39938,2130,0.71,64.1,60.0,0.0,0.00,0.0,SI2,F,Good


In [37]:
df.drop([560,15694,18395,39897,39938], axis = 0, inplace = True)

In [38]:
df.head()

,price,carat,depth,table,x,y,z,clarity,color,cut
0,3733,1.01,65.2,56.0,6.23,6.19,4.05,VS2,H,Fair
1,9452,1.51,65.0,64.0,7.10,7.03,4.59,VS2,H,Fair
2,12765,2.00,65.2,56.0,7.92,7.87,5.15,VS2,H,Fair
3,17713,1.99,60.5,63.0,8.20,8.11,4.93,VS2,H,Fair
4,2268,0.70,65.0,56.0,5.60,5.50,3.61,VS2,H,Fair


In [39]:
df[df['y'] == 0]

,price,carat,depth,table,x,y,z,clarity,color,cut


In [40]:
df[df['z'] == 0]

,price,carat,depth,table,x,y,z,clarity,color,cut
7436,5564,1.15,59.2,56.0,6.88,6.83,0.0,VS2,G,Ideal
17778,18207,2.02,62.7,53.0,8.02,7.95,0.0,VS2,H,Premium
18439,12631,2.18,59.4,61.0,8.49,8.45,0.0,SI2,H,Premium
18905,17265,2.20,61.2,59.0,8.42,8.37,0.0,SI1,H,Premium
19206,3167,1.01,58.1,59.0,6.66,6.60,0.0,I1,H,Premium
19990,15397,2.25,61.3,58.0,8.52,8.42,0.0,SI1,I,Premium
22744,3696,1.10,63.0,59.0,6.50,6.47,0.0,SI2,G,Premium
22926,3142,1.00,59.1,59.0,6.55,6.48,0.0,SI2,G,Premium
26174,3837,1.01,59.2,58.0,6.50,6.47,0.0,SI2,F,Premium
38574,18788,2.80,63.8,58.0,8.90,8.85,0.0,SI2,G,Good


In [41]:
df.drop([7436,17778,18439,18905,19206,19990,22744,22926,26174,38853], axis = 0, inplace = True)

In [42]:
df

,price,carat,depth,table,x,y,z,clarity,color,cut
0,3733,1.01,65.2,56.0,6.23,6.19,4.05,VS2,H,Fair
1,9452,1.51,65.0,64.0,7.10,7.03,4.59,VS2,H,Fair
2,12765,2.00,65.2,56.0,7.92,7.87,5.15,VS2,H,Fair
3,17713,1.99,60.5,63.0,8.20,8.11,4.93,VS2,H,Fair
4,2268,0.70,65.0,56.0,5.60,5.50,3.61,VS2,H,Fair
...,...,...,...,...,...,...,...,...,...,...
40450,3193,0.90,57.4,61.0,6.42,6.37,3.67,VVS2,J,Good
40451,15898,2.03,63.2,60.0,7.88,7.94,5.00,VVS2,J,Good
40452,666,0.38,63.4,56.0,4.56,4.62,2.91,VVS2,J,Good
40453,1945,0.91,63.1,57.0,6.06,6.12,3.84,I1,J,Good


In [43]:
df['price'].min()

326

In [44]:
df[df['price'] == 326]

,price,carat,depth,table,x,y,z,clarity,color,cut
12612,326,0.23,61.5,55.0,3.95,3.98,2.43,SI2,E,Ideal
24967,326,0.21,59.8,61.0,3.89,3.84,2.31,SI1,E,Premium


In [45]:
df.drop([12612,24967], axis = 0, inplace = True)

In [53]:
df

,price,carat,depth,table,x,y,z,clarity,color,cut
0,3733,1.01,65.2,56.0,6.23,6.19,4.05,VS2,H,Fair
1,9452,1.51,65.0,64.0,7.10,7.03,4.59,VS2,H,Fair
2,12765,2.00,65.2,56.0,7.92,7.87,5.15,VS2,H,Fair
3,17713,1.99,60.5,63.0,8.20,8.11,4.93,VS2,H,Fair
4,2268,0.70,65.0,56.0,5.60,5.50,3.61,VS2,H,Fair
...,...,...,...,...,...,...,...,...,...,...
40450,3193,0.90,57.4,61.0,6.42,6.37,3.67,VVS2,J,Good
40451,15898,2.03,63.2,60.0,7.88,7.94,5.00,VVS2,J,Good
40452,666,0.38,63.4,56.0,4.56,4.62,2.91,VVS2,J,Good
40453,1945,0.91,63.1,57.0,6.06,6.12,3.84,I1,J,Good


In [54]:
X = df.drop(columns=['price'])  
y = df['price'] 

cats = X.select_dtypes(include=['object']).columns.tolist()
cats

['clarity', 'color', 'cut']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
categorical_columns = ['clarity', 'color', 'cut']
for col in categorical_columns:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

In [59]:
# Crear matrices DMatrix para regresión
dtrain_reg = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

# Verificar las matrices DMatrix
print(dtrain_reg)
print(dtest_reg)

In [60]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}

# Entrenar el modelo
n = 100  # Número de rondas de boosting
model = xgb.train(params=params, dtrain=dtrain_reg, num_boost_round=n)

# Evaluar el modelo en el conjunto de prueba
predictions = model.predict(dtest_reg)
print(predictions)

[6509.1055   917.39307  436.1063  ... 1749.9775  6932.6626  2790.0862 ]


In [61]:

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

521.3042317939734

In [62]:
from sklearn.model_selection import train_test_split, GridSearchCV
model = xgb.XGBRegressor(tree_method='hist', enable_categorical=True, use_label_encoder=False, eval_metric='rmse')

# Definir el conjunto de hiperparámetros para buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Crear una instancia de GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)

In [65]:
# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

# Entrenar el modelo con los mejores hiperparámetros
best_model = grid_search.best_estimator_

# Hacer predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

Fitting 3 folds for each of 243 candidates, totalling 729 fits
Mejores hiperparámetros: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.9}
RMSE: 508.32504126659575


In [2]:
import joblib

In [68]:
# Guardar el modelo entrenado
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']

En otro notebook

In [3]:
# Cargar el modelo guardado
loaded_model = joblib.load('best_model.pkl')

In [19]:
X_test = pd.read_csv('../Diamonds_EDA-main/Competition_data/ih-datamadpt-0124-project-m-3/diamonds_test.csv')

In [20]:
X_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [21]:
X_test = X_test.drop('city', axis = 1)

In [22]:
X_test = X_test.drop('id', axis = 1)

In [23]:
X_test

,carat,cut,color,clarity,depth,table,x,y,z
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19
...,...,...,...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30
13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56
13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53
13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45


In [28]:
# Lista con el nuevo orden de las columnas
new_column_order = ['carat', 'depth', 'table', 'x', 'y', 'z', 'clarity', 'color', 'cut']

# Reindexar el DataFrame con el nuevo orden de las columnas
X_test = X_test.reindex(columns=new_column_order)

In [30]:
categorical_columns = ['clarity', 'color', 'cut']
for col in categorical_columns:
    X_test[col] = X_test[col].astype('category')

In [31]:
dtest = xgb.DMatrix(X_test, enable_categorical=True)

In [32]:
# Predecir en el nuevo conjunto de datos X_test
predictions = loaded_model.predict(X_test)

In [33]:
submission5 = pd.DataFrame(predictions, columns=['price'])
submission5 = submission5.reset_index()
column_nm = ["id", "price"]
submission5.columns = column_nm
submission5

,id,price
0,0,2848.012451
1,1,5662.229004
2,2,9549.805664
3,3,3943.817627
4,4,1630.473755
...,...,...
13480,13480,1669.317871
13481,13481,2446.734131
13482,13482,3161.668213
13483,13483,2157.821777


In [34]:
submission5.to_csv('submission5.csv', index = False)